In [15]:
import os,librosa
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers as opt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses
import pandas as pd
from sklearn.metrics import precision_score,recall_score,f1_score, cohen_kappa_score

In [61]:
seed=42
n_MFCC=6
column_names= ['MFCC_'+str(i) for i in range(1,n_MFCC+1)]

In [62]:
DATA_PATH="./Vowel_Data/Vowel/"

In [63]:
def get_mfcc(file_path):
    wave, sr= librosa.load(file_path, mono=True, sr=None)
    mfcc=librosa.feature.mfcc(wave,sr=sr,n_mfcc=n_MFCC)
    mfcc_df=pd.DataFrame(mfcc.T,columns=column_names) #convert array to pandas df
    return mfcc_df

def get_all_mfcc(path=DATA_PATH):
    labels=sorted(os.listdir(path))
    all_mfcc=pd.DataFrame() #empty dataframe to store all_mfccs
    
    for label_index,label in enumerate(labels):
        folder_mfcc=pd.DataFrame() #empty dataframe to store mfcc of indivuals vowels/words i.e. folders

        speech_files=[path + label + '/' + file for file in sorted(os.listdir(path+'/'+label))]
        for file in tqdm(speech_files,"Reading Speech of label -'{}'".format(label)):
            #print(file)
            individual_mfcc=get_mfcc(file_path=file)
            folder_mfcc=folder_mfcc.append(individual_mfcc,ignore_index=True)
        
        folder_mfcc['label']=label_index  #new columns for encoding label
        
        all_mfcc=all_mfcc.append(folder_mfcc,ignore_index=True)
        
    return all_mfcc

def get_train_test():
    all_mfccs=get_all_mfcc()
    
    y=all_mfccs['label'].values
    
    only_mfcc=all_mfccs.drop(labels=['label'],axis=1)
    standard_mfcc=(only_mfcc-only_mfcc.mean())/only_mfcc.std()
    X=standard_mfcc.values

    assert X.shape[0] == len(y)
    return X,y

In [64]:
X,y=get_train_test()

Reading Speech of label -'vowel7': 100%|██████████| 40/40 [00:00<00:00, 120.48it/s]


In [65]:
kfold=StratifiedKFold(n_splits=4,shuffle=True,random_state=seed)
cv_acc=[]
cv_AUC=[]
cv_f1=[]
cv_kappa=[]

for train,test in kfold.split(X,y):
    y_hot=to_categorical(y)
    
    model_v=Sequential()
    model_v.add(Dense(32, activation='tanh',input_shape=(X[train].shape[1],)))
    model_v.add(Dense(16, activation='tanh'))
    model_v.add(Dense(7, activation='softmax'))
    
    model_v.compile(
        optimizer=opt.Adam(learning_rate=0.005),
        loss='categorical_crossentropy',
        metrics=[['accuracy','AUC']]
    )
    model_v.fit(X[train], y_hot[train], epochs=50, batch_size=X[train].shape[0],verbose=0)
    
    scores=model_v.evaluate(X[test],y_hot[test],verbose=0)
    
    print("%s: %.2f" % (model_v.metrics_names[1],scores[1]))
    print("%s: %.2f" % (model_v.metrics_names[2],scores[2]))
    
    yhat_classes = model_v.predict_classes(X[test], verbose=0)
    
    f1 = f1_score(y[test], yhat_classes,average='weighted')
    print('F1: %.2f' % f1)
    
    kappa = cohen_kappa_score(y[test], yhat_classes)
    print('Cohens kappa: %.2f' % kappa)
    
    cv_acc.append(scores[1])
    cv_AUC.append(scores[2])
    cv_f1.append(f1)
    cv_kappa.append(kappa)
    
    print(' ')
    
print("Overall acc: %.2f (+/- %.2f)" % (np.mean(cv_acc),np.std(cv_acc)))
print("Overall AUC: %.2f (+/- %.2f)" % (np.mean(cv_AUC),np.std(cv_AUC)))
print("Overall f1: %.2f (+/- %.2f)" % (np.mean(cv_f1),np.std(cv_f1)))
print("Overall kappa: %.2f (+/- %.2f)" % (np.mean(cv_kappa),np.std(cv_kappa)))

accuracy: 0.60
auc: 0.92
F1: 0.59
Cohens kappa: 0.53
 
accuracy: 0.62
auc: 0.93
F1: 0.60
Cohens kappa: 0.56
 
accuracy: 0.60
auc: 0.92
F1: 0.58
Cohens kappa: 0.53
 
accuracy: 0.59
auc: 0.92
F1: 0.58
Cohens kappa: 0.52
 
Overall acc: 0.60 (+/- 0.01)
Overall AUC: 0.92 (+/- 0.00)
Overall f1: 0.59 (+/- 0.01)
Overall kappa: 0.54 (+/- 0.01)


In [66]:
save_specifier='standard_6mfcc_64_32_16_tanh_softmax_adam_0.005_batchfull'

In [67]:
cv_data_df=pd.DataFrame(data={'acc':cv_acc,
                              'AUC':cv_AUC,
                              'F1':cv_f1,
                              'Kappa':cv_kappa}
                       )
cv_data_df.to_csv('./New_without_CNN/vowel_cv_data_df_'+save_specifier+'.csv')

In [68]:
#WORD

In [69]:
DATA_PATH="./Word_Data/Word/"

In [70]:
def get_mfcc(file_path):
    wave, sr= librosa.load(file_path, mono=True, sr=None)
    mfcc=librosa.feature.mfcc(wave,sr=sr,n_mfcc=n_MFCC)
    mfcc_df=pd.DataFrame(mfcc.T,columns=column_names) #convert array to pandas df
    return mfcc_df

def get_all_mfcc(path=DATA_PATH):
    labels=sorted(os.listdir(path))
    all_mfcc=pd.DataFrame() #empty dataframe to store all_mfccs
    
    for label_index,label in enumerate(labels):
        folder_mfcc=pd.DataFrame() #empty dataframe to store mfcc of indivuals vowels/words i.e. folders

        speech_files=[path + label + '/' + file for file in sorted(os.listdir(path+'/'+label))]
        for file in tqdm(speech_files,"Reading Speech of label -'{}'".format(label)):
            #print(file)
            individual_mfcc=get_mfcc(file_path=file)
            folder_mfcc=folder_mfcc.append(individual_mfcc,ignore_index=True)
        
        folder_mfcc['label']=label_index  #new columns for encoding label
        
        all_mfcc=all_mfcc.append(folder_mfcc,ignore_index=True)
        
    return all_mfcc

def get_train_test():
    all_mfccs=get_all_mfcc()
    
    y=all_mfccs['label'].values
    
    only_mfcc=all_mfccs.drop(labels=['label'],axis=1)
    standard_mfcc=(only_mfcc-only_mfcc.mean())/only_mfcc.std()
    X=standard_mfcc.values

    assert X.shape[0] == len(y)
    return X,y

In [71]:
X,y=get_train_test()

Reading Speech of label -'Word7': 100%|██████████| 40/40 [00:00<00:00, 111.49it/s]


In [72]:
kfold=StratifiedKFold(n_splits=4,shuffle=True,random_state=seed)
cv_acc=[]
cv_AUC=[]
cv_f1=[]
cv_kappa=[]

for train,test in kfold.split(X,y):
    y_hot=to_categorical(y)
    
    model_w=Sequential()
    model_w.add(Dense(32, activation='tanh',input_shape=(X[train].shape[1],)))
    model_w.add(Dense(16, activation='tanh'))
    model_w.add(Dense(7, activation='softmax'))
    
    model_w.compile(
        optimizer=opt.Adam(learning_rate=0.005),
        loss='categorical_crossentropy',
        metrics=[['accuracy','AUC']]
    )
    model_w.fit(X[train], y_hot[train], epochs=50, batch_size=X[train].shape[0],verbose=0)
    
    scores=model_w.evaluate(X[test],y_hot[test],verbose=0)
    
    print("%s: %.2f" % (model_w.metrics_names[1],scores[1]))
    print("%s: %.2f" % (model_w.metrics_names[2],scores[2]))
    
    yhat_classes = model_w.predict_classes(X[test], verbose=0)
    
    f1 = f1_score(y[test], yhat_classes,average='weighted')
    print('F1: %.2f' % f1)
    
    kappa = cohen_kappa_score(y[test], yhat_classes)
    print('Cohens kappa: %.2f' % kappa)
    
    cv_acc.append(scores[1])
    cv_AUC.append(scores[2])
    cv_f1.append(f1)
    cv_kappa.append(kappa)
    
    print(' ')
    
print("Overall acc: %.2f (+/- %.2f)" % (np.mean(cv_acc),np.std(cv_acc)))
print("Overall AUC: %.2f (+/- %.2f)" % (np.mean(cv_AUC),np.std(cv_AUC)))
print("Overall f1: %.2f (+/- %.2f)" % (np.mean(cv_f1),np.std(cv_f1)))
print("Overall kappa: %.2f (+/- %.2f)" % (np.mean(cv_kappa),np.std(cv_kappa)))

accuracy: 0.43
auc: 0.81
F1: 0.41
Cohens kappa: 0.33
 
accuracy: 0.42
auc: 0.81
F1: 0.39
Cohens kappa: 0.32
 
accuracy: 0.43
auc: 0.82
F1: 0.41
Cohens kappa: 0.33
 
accuracy: 0.41
auc: 0.80
F1: 0.39
Cohens kappa: 0.31
 
Overall acc: 0.42 (+/- 0.01)
Overall AUC: 0.81 (+/- 0.00)
Overall f1: 0.40 (+/- 0.01)
Overall kappa: 0.32 (+/- 0.01)


In [60]:
cv_data_df=pd.DataFrame(data={'acc':cv_acc,
                              'AUC':cv_AUC,
                              'F1':cv_f1,
                              'Kappa':cv_kappa}
                       )
cv_data_df.to_csv('./New_without_CNN/vowel_cv_data_df_'+save_specifier+'.csv')